In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3, InceptionV3, ResNet101, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os

In [ ]:
!mkdir dataset

In [ ]:
!unzip /content/drive/MyDrive/data/data.zip -d /content/dataset

In [ ]:
train_path = '/content/dataset'
labels_list = os.listdir(train_path)
img_size = 224
img_shape = (img_size,img_size,3)
batch_size = 32
epochs = 10
dropout_rate = 0.5
num_of_predict = len(labels_list)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2866 images belonging to 12 classes.


In [ ]:
efficientnet = EfficientNetB3(weights="/content/drive/MyDrive/data/efficientnetb3_notop.h5",
                              include_top=False,
                              input_shape=img_shape,
                              drop_connect_rate=dropout_rate)
inputs = Input(shape=img_shape)
efficientnet = efficientnet(inputs)
pooling = layers.GlobalAveragePooling2D()(efficientnet)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(num_of_predict, activation="softmax")(dropout)
eff_model = Model(inputs=inputs, outputs=outputs)

eff_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 7, 7, 1536)        10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                18444     
Total params: 10,801,979
Trainable params: 10,714,676
Non-trainable params: 87,303
_________________________________________________________________


In [ ]:
decay_steps = int(round(2866/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)
eff_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])

In [ ]:
history = eff_model.fit(train_generator,
                       epochs = 10)

Epoch 1/10
 4/90 [>.............................] - ETA: 1:02 - loss: 2.7142 - accuracy: 0.0768

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


90/90 [==============================] - 110s 723ms/step - loss: 2.3284 - accuracy: 0.2179
Epoch 2/10
90/90 [==============================] - 64s 708ms/step - loss: 1.1714 - accuracy: 0.6766
Epoch 3/10
90/90 [==============================] - 64s 708ms/step - loss: 0.6676 - accuracy: 0.8202
Epoch 4/10
90/90 [==============================] - 64s 704ms/step - loss: 0.4118 - accuracy: 0.8798
Epoch 5/10
90/90 [==============================] - 64s 704ms/step - loss: 0.3475 - accuracy: 0.9023
Epoch 6/10
90/90 [==============================] - 64s 707ms/step - loss: 0.2696 - accuracy: 0.9312
Epoch 7/10
90/90 [==============================] - 64s 708ms/step - loss: 0.2472 - accuracy: 0.9324
Epoch 8/10
90/90 [==============================] - 64s 703ms/step - loss: 0.1974 - accuracy: 0.9502
Epoch 9/10
90/90 [==============================] - 64s 704ms/step - loss: 0.1486 - accuracy: 0.9594
Epoch 10/10
90/90 [==============================] - 65s 716ms/step - loss: 0.1705 - accuracy: 0.9485

In [ ]:
eff_model.evaluate(train_generator)

21/90 [======>.......................] - ETA: 37s - loss: 0.1723 - accuracy: 0.9524

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


90/90 [==============================] - 49s 525ms/step - loss: 0.1714 - accuracy: 0.9532


[0.17139869928359985, 0.9532449245452881]

In [ ]:
eff_model.save('/content/eff_tmp.h5')

In [ ]:
test_model = tf.keras.models.load_model('/content/eff_tmp.h5')

In [ ]:
test_model.evaluate(train_generator)

 6/90 [=>............................] - ETA: 48s - loss: 0.1508 - accuracy: 0.9740

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


90/90 [==============================] - 50s 533ms/step - loss: 0.1753 - accuracy: 0.9491


[0.17527298629283905, 0.949057936668396]

In [ ]:
# eff_model.save('/content/drive/MyDrive/data/eff_final.h5')